# Ensemble

In [9]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from data import train, validation
from data import X, y, categorical, numerical

In [36]:
pred_train = pd.read_csv('predictions/train.csv', index_col=0)
pred_validation = pd.read_csv('predictions/validation.csv', index_col=0)

In [37]:
pred_train

,0,1,2,3,4,7,10,13
0,262.264045,208.946981,224.181583,276.375883,287.523662,288.660298,224.181583,208.887056
1,5.717018,2.380884,-4.723311,4.678724,5.449531,4.829065,-4.723311,1.475746
2,18.588675,20.537849,23.338251,15.766221,14.711933,17.162290,23.338251,20.961534
3,8.092857,37.149286,28.099011,20.377933,19.100246,19.010970,28.099011,37.406971
4,25.287449,32.700862,28.449217,25.734911,26.616290,25.761958,28.449217,32.706583
...,...,...,...,...,...,...,...,...
1987,10.654115,14.760095,10.972617,17.758581,17.204515,17.541649,10.972617,14.583920
1988,70.258385,82.341273,81.249817,63.575912,63.284170,66.195904,81.249817,82.850616
1989,5.948459,16.275653,19.949134,5.937480,5.756618,5.955374,19.949134,16.972041
1990,28.373535,29.719892,31.653332,26.640663,25.384817,25.602749,31.653332,29.598150


In [14]:
MLPRegressor?

Init signature:
MLPRegressor(
    hidden_layer_sizes=(100,),
    activation='relu',
    *,
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    power_t=0.5,
    max_iter=200,
    shuffle=True,
    random_state=None,
    tol=0.0001,
    verbose=False,
    warm_start=False,
    momentum=0.9,
    nesterovs_momentum=True,
    early_stopping=False,
    validation_fraction=0.1,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    n_iter_no_change=10,
    max_fun=15000,
)
Docstring:     
Multi-layer Perceptron regressor.

This model optimizes the squared-loss using LBFGS or stochastic gradient
descent.

.. versionadded:: 0.18

Parameters
----------
hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
    The ith element represents the number of neurons in the ith
    hidden layer.

activation : {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
    Activation function for the hidden layer.

    - 'id

In [66]:
VotingRegressor?

Init signature: VotingRegressor(estimators, *, weights=None, n_jobs=None, verbose=False)
Docstring:     
Prediction voting regressor for unfitted estimators.

A voting regressor is an ensemble meta-estimator that fits several base
regressors, each on the whole dataset. Then it averages the individual
predictions to form a final prediction.

Read more in the :ref:`User Guide <voting_regressor>`.

.. versionadded:: 0.21

Parameters
----------
estimators : list of (str, estimator) tuples
    Invoking the ``fit`` method on the ``VotingRegressor`` will fit clones
    of those original estimators that will be stored in the class attribute
    ``self.estimators_``. An estimator can be set to ``'drop'`` using
    ``set_params``.

    .. versionchanged:: 0.21
        ``'drop'`` is accepted. Using None was deprecated in 0.22 and
        support was removed in 0.24.

weights : array-like of shape (n_regressors,), default=None
    Sequence of weights (`float` or `int`) to weight the occurrences of

In [73]:
ElasticNet?

Init signature:
ElasticNet(
    alpha=1.0,
    *,
    l1_ratio=0.5,
    fit_intercept=True,
    normalize=False,
    precompute=False,
    max_iter=1000,
    copy_X=True,
    tol=0.0001,
    warm_start=False,
    positive=False,
    random_state=None,
    selection='cyclic',
)
Docstring:     
Linear regression with combined L1 and L2 priors as regularizer.

Minimizes the objective function::

        1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

If you are interested in controlling the L1 and L2 penalty
separately, keep in mind that this is equivalent to::

        a * L1 + b * L2

where::

        alpha = a + b and l1_ratio = a / (a + b)

The parameter l1_ratio corresponds to alpha in the glmnet R package while
alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio
= 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable,
unless you supply your own sequence of alpha.

Read

In [114]:
LinearRegression?

Init signature:
LinearRegression(
    *,
    fit_intercept=True,
    normalize=False,
    copy_X=True,
    n_jobs=None,
    positive=False,
)
Docstring:     
Ordinary least squares Linear Regression.

LinearRegression fits a linear model with coefficients w = (w1, ..., wp)
to minimize the residual sum of squares between the observed targets in
the dataset, and the targets predicted by the linear approximation.

Parameters
----------
fit_intercept : bool, default=True
    Whether to calculate the intercept for this model. If set
    to False, no intercept will be used in calculations
    (i.e. data is expected to be centered).

normalize : bool, default=False
    This parameter is ignored when ``fit_intercept`` is set to False.
    If True, the regressors X will be normalized before regression by
    subtracting the mean and dividing by the l2-norm.
    If you wish to standardize, please use
    :class:`~sklearn.preprocessing.StandardScaler` before calling ``fit``
    on an estimator wi

In [120]:
from utils import StandardizedGridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet

meta = StandardizedGridSearchCV(
    estimator = LinearRegression(),
    param_grid = {
        'positive': [True, False],
    },
)

meta.fit(pred_train, train[y])
meta.results

Fitting 10 folds for each of 2 candidates, totalling 20 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,split3_test_neg_mean_squared_error,split4_test_neg_mean_squared_error,split5_test_neg_mean_squared_error,...,split2_train_r2,split3_train_r2,split4_train_r2,split5_train_r2,split6_train_r2,split7_train_r2,split8_train_r2,split9_train_r2,mean_train_r2,std_train_r2
positive,,,,,,,,,,,,,,,,,,,,,
False,0.003720,0.000489,0.003061,0.000625,-160.506543,-121.297027,-258.825803,-222.788704,-186.978366,-87.785727,...,0.988124,0.987878,0.987549,0.986997,0.985409,0.986592,0.987305,0.986770,0.987083,0.000719
True,0.003862,0.000717,0.002960,0.000259,-278.591319,-188.218864,-639.538413,-325.056762,-529.342387,-198.907090,...,0.968967,0.965364,0.967443,0.965278,0.963262,0.963793,0.976875,0.963402,0.966318,0.003917


In [122]:
meta.results[['mean_train_neg_mean_squared_error', 'mean_test_neg_mean_squared_error']]

,mean_train_neg_mean_squared_error,mean_test_neg_mean_squared_error
positive,,
False,-131.945207,-238.226126
True,-345.861807,-668.911612


In [123]:
meta.save('final_ensemble.p')

# Model evaluation

The ensemble model was evaluated against the best individual model on the validation set. The ensemble model shows a significant improvement in prediction variance with a small improvement in prediction bias. 

In [145]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

y_true = validation[y]

print('Best indivdual model:')
y_pred = pred_validation['0']
print('- mse =', mean_squared_error(y_true=y_true, y_pred=y_pred))
print('- mae =', mean_absolute_error(y_true=y_true, y_pred=y_pred))
print('- std =', (y_true - y_pred).std())

print()
print('Ensemble model:')
y_pred = meta.predict(pred_validation)
print('- mse =', mean_squared_error(y_true=y_true, y_pred=y_pred))
print('- mae =', mean_absolute_error(y_true=y_true, y_pred=y_pred))
print('- std =', (y_true - y_pred).std())

Best indivdual model:
- mse = 1265.690846715096
- mae = 18.181575382899513
- std = 125.55516520789725

Ensemble model:
- mse = 1238.1999837840451
- mae = 16.91038750158237
- std = 35.19033864842475
